[View in Colaboratory](https://colab.research.google.com/github/iamrajee/Data-Analysis/blob/master/fold.ipynb)

https://github.com/tensorflow/fold/blob/master/tensorflow_fold/g3doc/quick.ipynb

https://medium.com/@maheshwar.ligade/dynamic-computation-graphs-dcg-with-tensorflow-fold-33638b2d5754

In [10]:
!pip install https://storage.googleapis.com/tensorflow_fold/tensorflow_fold-0.0.1-py3-none-linux_x86_64.whl

In [11]:
!python -c 'import tensorflow_fold'

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_fold/__init__.py", line 22, in <module>
    from tensorflow_fold.blocks.block_compiler import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_fold/blocks/block_compiler.py", line 28, in <module>
    from tensorflow_fold.blocks import loom_ops
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_fold/blocks/loom_ops.py", line 19, in <module>
    import tensorflow_fold.blocks.result_types as tdt
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_fold/blocks/result_types.py", line 27, in <module>
    from tensorflow_fold.public import lo

In [7]:
import random
import tensorflow as tf
sess = tf.InteractiveSession()
import tensorflow_fold as td

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


NotFoundError: ignored

In [5]:
scalar_block = td.Scalar()
vector3_block = td.Vector(3)

NameError: ignored

In [0]:
def block_info(block):
    print("%s: %s -> %s" % (block, block.input_type, block.output_type))
    
block_info(scalar_block)
block_info(vector3_block)


In [0]:
scalar_block.eval(42)

In [0]:
vector3_block.eval([1,2,3])

In [0]:
record_block = td.Record({'foo': scalar_block, 'bar': vector3_block})
block_info(record_block)

In [0]:
record_block.eval({'foo': 1, 'bar': [5, 7, 9]})

In [0]:
record2vec_block = record_block >> td.Concat()
record2vec_block.eval({'foo': 1, 'bar': [5, 7, 9]})

In [0]:
negative_block = record2vec_block >> td.Function(tf.negative)
negative_block.eval({'foo': 1, 'bar': [5, 7, 9]})

In [0]:
map_scalars_block = td.Map(td.Scalar())

In [0]:
block_info(map_scalars_block)

In [0]:
jagged_block = td.Map(td.Map(td.Scalar()))
block_info(jagged_block)

In [0]:
seq_of_tuples_block = td.Map(td.Record({'foo': td.Scalar(), 'bar': td.Scalar()}))
seq_of_tuples_block.eval([{'foo': 1, 'bar': 2}, {'foo': 3, 'bar': 4}])

In [0]:
tuple_of_seqs_block = td.Record({'foo': td.Map(td.Scalar()), 'bar': td.Map(td.Scalar())})
tuple_of_seqs_block.eval({'foo': range(3), 'bar': range(7)})

In [0]:
((td.Map(td.Scalar()) >> td.Sum()).eval(range(10)),
 (td.Map(td.Scalar()) >> td.Min()).eval(range(10)),
 (td.Map(td.Scalar()) >> td.Max()).eval(range(10)))

In [0]:
(td.Map(td.Scalar()) >> td.Reduce(td.Function(tf.multiply))).eval(range(1,10))

In [0]:
((td.Map(td.Scalar()) >> td.Fold(td.Function(tf.divide), tf.ones([]))).eval(range(1,5)),
 (td.Map(td.Scalar()) >> td.Reduce(td.Function(tf.divide), tf.ones([]))).eval(range(1,5)))  # bad, not associative!

In [0]:
def reduce_net_block():
    net_block = td.Concat() >> td.FC(20) >> td.FC(1, activation=None) >> td.Function(lambda xs: tf.squeeze(xs, axis=1))
    return td.Map(td.Scalar()) >> td.Reduce(net_block)

In [0]:
def random_example(fn):
    length = random.randrange(1, 10)
    data = [random.uniform(0,1) for _ in range(length)]
    result = fn(data)
    return data, result

In [0]:
random_example(sum)

In [0]:
random_example(min)

In [0]:
def train(fn, batch_size=100):
    net_block = reduce_net_block()
    compiler = td.Compiler.create((net_block, td.Scalar()))
    y, y_ = compiler.output_tensors
    loss = tf.nn.l2_loss(y - y_)
    train = tf.train.AdamOptimizer().minimize(loss)
    sess.run(tf.global_variables_initializer())
    validation_fd = compiler.build_feed_dict(random_example(fn) for _ in range(1000))
    for i in range(2000):
        sess.run(train, compiler.build_feed_dict(random_example(fn) for _ in range(batch_size)))
        if i % 100 == 0:
            print(i, sess.run(loss, validation_fd))
    return net_block

In [0]:
sum_block = train(sum)

In [0]:
sum_block.eval([1, 1])

In [0]:
min_block = train(min)

In [0]:
min_block.eval([2, -1, 4])

In [0]:
min_block.eval([0.3, 0.2, 0.9])